In [31]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, qty)
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    # load data.
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # shuffle an index vector
    idx = Vector(1:size(pts_pcs,1))
    rng = MersenneTwister(1234)
    idx = shuffle(rng, idx)
    
    # shuffle input pts and convert them to spherical
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]))

    # generate scan data
    scan = scan_tour(sph)
    
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
    
    # save
    writecsv("scan1.csv", scan)
end

test_transforms()
test_scan()

8.881784197001252e-16

51-element Array{Int64,1}:
 32
 39
 13
 11
 17
 40
 42
 51
  1
 31
 38
 27
 44
  ⋮
 46
 35
  4
 10
 16
  3
 20
 29
 23
 12
 24
 45

51×3 Array{Float64,2}:
  0.576372  1.28479    2.18214 
  0.363344  0.944167   1.24128 
 -0.820859  0.787889   0.985161
  0.535124  1.28018    2.25283 
  0.576315  1.28427    2.18212 
  0.599436  1.28617    2.15049 
  0.626194  1.28575    2.13824 
  0.652517  1.29513    2.08841 
  0.767131  1.30497    1.96929 
  0.766731  1.32892    1.80089 
  0.766751  1.33604    1.65115 
  0.289392  1.39443    1.07905 
  0.110988  1.39614    1.08797 
  ⋮                            
  0.508046  0.969538   2.262   
  0.782297  0.949731   1.86634 
  0.780002  0.949594   1.79784 
  0.511743  1.38645    1.44195 
  0.607771  1.39611    1.50164 
  0.607382  1.39593    1.37527 
  0.576386  1.39999    1.43868 
  0.784571  1.31049    0.89574 
  0.784553  1.35965    0.868719
  0.783446  1.39171   -0.171378
  0.785806  1.38555   -0.202632
  0.65748   0.980296   2.04134 

51×3 Array{Float64,2}:
  0.8247    0.735601  0.899041
  0.507975  0.96955   2.29049 
  0.110988  1.39614   1.08797 
  0.766751  1.33604   1.65115 
  0.82286   1.32168   1.3639  
  0.508046  0.969538  2.262   
  0.780002  0.949594  1.79784 
  0.65748   0.980296  2.04134 
  0.576372  1.28479   2.18214 
  0.824679  0.735608  0.932535
  0.15167   1.26864   2.30922 
  0.394262  1.72072   0.351288
  0.607771  1.39611   1.50164 
  ⋮                           
  0.576386  1.39999   1.43868 
  0.091392  1.73044   0.370374
  0.535124  1.28018   2.25283 
  0.766731  1.32892   1.80089 
  0.822105  1.27383   1.2575  
 -0.820859  0.787889  0.985161
  0.818726  1.1351    1.11076 
  0.821283  0.787891  0.98518 
  0.777238  1.48397   0.777762
  0.289392  1.39443   1.07905 
  0.748298  1.54939   0.668686
  0.607382  1.39593   1.37527 

(12146, 3)
